In [ ]:
# import all the packages needed
import urllib.request
import urllib.error
import json
import datetime
import time
import sys, os
import logging
import glob
from urllib.error import HTTPError
import configparser
from configparser import SafeConfigParser

In [ ]:
# function to convert data to json format 
def convert(input):
    if isinstance(input, dict):
        return {convert(key): convert(value) for key, value in input.items()}
    elif isinstance(input, list):
        return [convert(element) for element in input]
    else:
        return input

In [ ]:
# function to get all the imdb movie ID's
def getIMDBIDs(script_dir):
    configFiles = glob.glob(script_dir + '\Data\Movie Details\*.json')
    imdb_ids =[]
    for file in configFiles:
        with open (file, encoding='utf-8') as json_file:
            json_data = json.load(json_file)
            imdbid = json_data['imdb_id']
            imdb_ids.append(imdbid)
    return imdb_ids

In [ ]:
# reading all the imdb movies to get a list of ID's
configFiles = glob.glob(os.getcwd() + '\Data\IMDB Movies\*.json')
a = []
for file in configFiles:
    b = os.path.basename(file)
    c = os.path.splitext(b)[0]
    a.append(c)

In [ ]:
# getting all the movie details in the required format as belowby passing imdb id to the api link
def getIMDBMovieDetails():
    config = SafeConfigParser()
    script_dir = os.getcwd() 
    config_file = os.path.join(script_dir, 'config\config.cfg')
    config.read(config_file)
    imdb_json_file_path = config.get('files','imdb_json_file_path')
    api_key = config.get('imdb','api_key') 
    ids = getIMDBIDs(script_dir)
    # iterating through the IMDB movie ID's
    for ID in ids:
        # checking whether ie ID is empty or not
        if ID not in a and (ID != "") and (ID != None):
            try:
                request_string = "http://www.myapifilms.com/imdb/idIMDB?idIMDB="+str(ID)+"&token="+str(api_key)+"&format=json&language=en-us&aka=0&business=1&seasons=1&seasonYear=1&technical=1&trailers=0&movieTrivia=0&awards=1&moviePhotos=0&movieVideos=0&actors=1&biography=1&uniqueName=0&filmography=0&bornAndDead=0&starSign=0&actorActress=0&actorTrivia=0&similarMovies=0&goofs=0&keyword=1&quotes=0&fullSize=1&companyCredits=1&filmingLocations=1"
                print(request_string)
                response = urllib.request.urlopen(request_string)
                content = response.read().decode('utf-8')
                if content:
                    movies = convert(json.loads(content))
                    if("data" in movies):
                        if len(movies['data']['movies']) >= 1:
                            json_file_name = (str(ID)+'.json')
                            json_file_name = "".join([imdb_json_file_path,'\\', json_file_name])
                            json_file = open(json_file_name, 'w', encoding='utf-8')
                            json_file.write(content)
                            json_file.close() 
                else:
                    return
                time.sleep(3)
            except HTTPError as e:
                if e.code == 404:
                    print("Page is not found")
                    continue
                if e.code == 429:
                    print ("Waiting. You've reached an API limit for the day.")
                    time.sleep(30) # wait 30 seconds and try again
                    break

In [ ]:
getIMDBMovieDetails()